This file is part of GaPSE
Copyright (C) 2022 Matteo Foglieni

GaPSE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

GaPSE is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with GaPSE. If not, see <http://www.gnu.org/licenses/>.
```


```

# Generic Window

This is a small tutorial in order to show how to use GaPSE when one is interested in the power spectrum obtained with a generic window function, and not with the toy-model we implemented.
The theoretical formula we will use is the following (Eq.(2.13) of Castorina, Di Dio (2021)); we remember that all the distances $s, s_1, s_2$ are comoving distances measured in $h_0^{-1}\mathrm{Mpc}$:

$$
    \left\langle \hat{P}_L(k) \right\rangle = 
        \frac{2 L + 1}{A} (-i)^L
        \sum_{\ell=0}^{\infty} 
        \sum_{\ell_1=0}^{\infty} 
        \begin{pmatrix}
            L & \ell & \ell_1 \\
            0 & 0 & 0
        \end{pmatrix}^2
        \int_0^{\infty}\!\mathrm{d} s \, s^2 \, \xi_\ell(s, s_{\rm eff}) \, 
        j_L(k s) \, Q_{\ell_1}(s) \, ,
$$

where:
- $\left\langle \hat{P}_L(k) \right\rangle$ is the order $L$ Power Spectrum of the effect we are interested in; we are basing this expression on the Yamamoto estimator (see Yamamoto (2000) and Yamamoto (2006))
- $A$ is a normalization constant
- the 2x3 matrix represents the Wigner-3j symbols
- $\xi_\ell$ is the order $\ell$ multipole of the Two-Point Correlation Function (TPCF)
- $j_L$ is the spherical Bessel function of order $L$
- $s_{\mathrm{eff}}$ is the comoving distance associated with the effective redshift (see the `TUTORIAL.ipynb` notebook)

$Q_{\ell_1}$ can be easily estimated with FFT methods:

$$
\begin{split}
    Q_{\ell_1}(s) &:= \int_0^{\infty}\mathrm{d}s_1 \; s_1^2 \;
    \phi(s_1) \; F_{\ell_1}(s_1, s) \\
    F_{\ell_1} (s_1 , s) &:= 
    \int_{4\pi} \mathrm{d}\Omega_{\mathbf{\hat{s}}} \,  
    \int_{4\pi} \mathrm{d}\Omega_{\mathbf{\hat{s}}_1} \,
    \phi(s_2) \, W(\mathbf{\hat{s}}_1) \, W(\mathbf{\hat{s}}_2) \,
    \mathcal{L}_{\ell_1}(\mathbf{\hat{s}} \cdot \mathbf{\hat{s}}_1)  \, .
\end{split}
$$

where:
- $\mathcal{L}_{\ell_1}$ is the Legendre polynomial of order $\ell_1$
- $\mathrm{d}\Omega_{\mathbf{\hat{s}}}$ is the infinitesimal solid angle pointing in the direction of the versor $\mathbf{\hat{s}}$
- $\phi(s)$ and  $W(\mathbf{\hat{s}})$ are respectively the radial and angular part of your window function (we remember that we assumed that such separability of the window function is possible)

Suppose we want the Power Spectrum of the Galaxy Number Counts (GNC) General Relatvistic (GR) effects. The idea is simple:
- first of all, you must compute the $Q_{\ell_1}(s)$ multipoles ($0 \leq \ell_1 \leq 4$ should be enough in almost all the realistic cases, but depending on your specific choise maybe you could even need less than that) and store them in a file let's call it `Ql_multipoles.txt`.  The file should have the following structure:
```text
    $ cat Ql_multipoles.txt 
    # Any comment line in the file must start with a #
    # you can have how many comment lines you want in the header; they 
    # will be all skipped.
    # Then you must provide in blank space separated columns:
    # - as first column, the comoving distance values, measured in Mpc/h_0
    # - from the second column onwards, all the Q_{\ell_1} multipoles you want;
    #   they must be ordered followinf the ascending multipole order (so \ell_1 = 0
    #   must be the 2 column), and you can go as further as you want in the multipole
    #   order
    # 
    # s [Mpc/h_0] 	 Q_{l1=0} 	 Q_{l1=1} 	 Q_{l1=2} 	 ...
    1.0 	0.9999999999999999 	 1.445269850978461e-7 	 0.000011917268941324522	...
    21.0	0.9832914433168294 	 -0.0025537781362117177  -0.0033199998619560947	    ...
    41.0	0.9669175943095181 	 -0.004923364937797496 	 -0.006463561496567318	    ... 	
    ... 	... 				 ... 					 ...
```

- then you can proceed in the computation of all the GNC Two-Point Correlation Functions (TPCFs) for the same multipole orders, setting `use_windows=false`
- finally, you'll use the functions `create_file_for_XiMultipole` and `PS_multipole_GenWin` , and you'll get the PS multipole described in the previous equation.